# Coverage Report Generation V2.0

## 1. Variables to modify

In [1]:
user = 'jmarinbe@cisco.com' # Your Cisco e-mail address
cr_tableau_template_name = 'C:/Users/jmarinbe/OneDrive - Cisco/Cisco Tableau/CR Template.twb' # twb template name/path
personal_access_token_name = 'juanma'
personal_access_token_secret = 'Sq1X69ihR+Wy+CgkQQCVYQ==:2e30VjYlJu9SZp0goiTZh34coeC9Avk6'

## 2. Libraries & Functions

In [2]:
%%time
# ----------------------------------------- LIBRARIES

#!pip install snowflake-connector-python
#!pip install tableauhyperapi
#!pip install tableau-api-lib
#!pip install tableauserverclient
#!pip install hyperapi
#!pip install pyarrow
#!pip install fastparquet

import os
import datetime
import pandas as pd
import numpy as np
import itertools
import shutil
import xml.etree.ElementTree as ET
import snowflake.connector
import re
import webbrowser

from zipfile import ZipFile
import tableauserverclient as TSC
from tableauhyperapi import *

import smartsheet_lib as smartsheet
import utils_coverage as utilsc

# ----------------------------------------- FUNCTIONS

qs = {8:1,9:1,10:1,11:2,12:2,1:2,2:3,3:3,4:3,5:4,6:4,7:4}
fy = {1:1,2:1,3:0,4:0}

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

Wall time: 2.1 s


## 3. Generating the Workbooks

In [3]:
%%time
    
current_path = os.getcwd().replace("\\","/")

date = datetime.datetime.today()
date = date.date()

if date.weekday() in [0,1]:
    delta_t = str(date-datetime.timedelta(4))
else:
    delta_t = str(date-datetime.timedelta(1))
    
month = datetime.datetime.today().strftime("%B")

print ('Current Path: ' + str(current_path))
print ('Current Date: ' + str(date))

# -----------------------------------------------------------------

smartsheet_client = smartsheet.init_conn("J937aTx1hHKkv3ju15G0XGC0Qz6af50MNrhLq")
oa_sheet = smartsheet.load_sheet(7190965472520068,client=smartsheet_client,modified_since=delta_t)
oa_df = pd.DataFrame()
sheet_new = smartsheet.get_last_n_rows(oa_sheet,n_rows=3000)
oa_df = smartsheet.sheet_to_df2(sheet_new,columns=oa_sheet.columns)
oa_df = oa_df.query("`Request ID` != ''")
oa_df['Request ID'] = oa_df['Request ID'].apply(lambda x:int(x)) # drop request id decimal places

fields_df = utilsc.get_da_requests(da=user,df=oa_df)
fields_df['CR Creation Date'] = date
fields_df

Current Path: C:/Users/jmarinbe/OneDrive - Cisco/Cisco Tableau
Current Date: 2022-03-10
Loaded Sheet: OP Intake Request Fields FY22Q3
Rows: 858
Wall time: 22 s


,index,Request ID,Date Created,Assigned DA,Campaign Name,Customer Name,Input file URL,ID TYPE,SAV ID,CAV ID,...,Cov Rejected Reason,Cov URL,Cov Hold Reason,Cov Uploaded Time,sav_list,gu_list,cav_list,cr_list,contract_list,CR Creation Date
0,17,112586,2022-02-16 18:34:00.000,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,SFDC US,,SAV ID,283391247,,...,,,,,[283391247],"[7000, 145033684, 3423363, 59606, 154774894, 2...",[],[],[],2022-03-10
1,18,112590,2022-02-16 18:35:00.000,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,MICROSOFT EMEAR,,SAV ID,283347799,,...,,,,,[283347799],"[211989756, 2909501, 63503254, 8983413, 38913,...",[],[],[],2022-03-10
2,704,115463,2022-03-09 05:50:37.659,jmarinbe@cisco.com,Reactive,Avis Budget Group,,SAV ID,276737267,,...,,https://cx-tableau-stage.cisco.com/#/site/Comp...,,,[276737267],[30055],[],[2376649],[],2022-03-10


In [4]:
%%time
# Retrieving snowflake data and creating dataframes
savs_list,gu_list,cav_list,cr_list = utilsc.get_ids_list(fields_df,separator=',') # IDs to look for on Snowflake
uncovered_data = utilsc.get_uncovered_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list)  # Retrieve Uncovered Data from SF
coverage_data = utilsc.get_coverage_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list) # Retrieve Coverage Data from SF
contracts_data = utilsc.get_contracts_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list)  # Retrieve Contracts Data from SF
df_uncovered,df_coverage,df_contracts = utilsc.format_columns(uncovered_data,coverage_data,contracts_data) # Rename columns to match tableau extracts

Wall time: 17.8 s


In [5]:
%%time
sns_df = pd.DataFrame()
folders_list = []
smartsheet_fields = ['index','Request ID','Date Created',
                         'Assigned DA','Campaign Name','Customer Name',
                         'Input file URL','ID TYPE','SAV ID','CAV ID',
                         'CAV BU ID','GU ID','Lvl1','Lvl2 (Region)',
                         'Contract ID','Inventory Name','Appliance ID',
                         'CR Party Name','CR Party ID','Comments','DA Comments',
                         'Status','Requester Name','Who should be notified on completion of Analysis',
                         'OP Status','CR Creation Date']
for idx in range(len(fields_df)):
    name = re.sub('[^A-Za-z0-9\-]+', '', fields_df['Customer Name'][idx][0:15])
    theater = fields_df['Lvl1'][idx]
    req_type = fields_df['ID TYPE'][idx]
    
    if req_type == 'SAV ID':
        ids = fields_df['sav_list'][idx]
        id_type = 'SAV'
    elif req_type == 'GU ID':
        ids = fields_df['gu_list'][idx]
        id_type = 'GU'
    elif req_type == 'CR Party ID':
        ids = fields_df['cr_list'][idx]
        id_type = 'CR'
    elif req_type == 'CAV ID':
        ids = fields_df['cav_list'][idx]
        id_type = 'CAV'
        
    req_id = fields_df['Request ID'][idx]
    date_created = fields_df['CR Creation Date'][idx]
    print(f"{idx+1}/{len(fields_df)}: {name}-{req_id}",end=" - ")
    
    folder =  f"CR_{theater}_{req_id}_{name}_{req_type}_{str(date.year)}_{str(date.month)}_{str(date.day)}" # folder name
    folder_path = f"{current_path}/CR/{month}/{date}/{folder}"    
      
    # -------------------------------------------------- Creating the extracts   
    uncovered_data_filtered = df_uncovered[df_uncovered['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
    uncovered_data_filtered = uncovered_data_filtered[uncovered_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
    if len(uncovered_data_filtered) == 0:
        print(bcolors.FAIL + f"No Uncovered Oppty - No Serial Numbers to upload" + bcolors.ENDC)
        folders_list.append("N/A")
        
    else:
        try:
            os.makedirs(f"{folder_path}/Data")
        except: pass    
        folders_list.append(folder)
        
        coverage_data_filtered = df_coverage[df_coverage['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
        coverage_data_filtered = coverage_data_filtered[coverage_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
        contracts_data_filtered = df_contracts[contracts_data['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
        contracts_data_filtered = contracts_data_filtered[contracts_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
        smartsheet_filtered = fields_df[smartsheet_fields].iloc[[idx]]
        
        # Removing some columns
        uncovered_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True)
        coverage_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True) 
        contracts_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True)   

        uncovered_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/uncovered.csv")
        coverage_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/contracts-and-coverage.csv")
        contracts_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/contract-view.csv")
        smartsheet_filtered.to_csv(f"{folder_path}/Data/Smartsheet.csv")
        
        
        uncovered_data_filtered.head()
        

        # -------------------------------------------------- Creating the workbooks twb
        """tree = ET.parse(cr_tableau_template_name)
        for extract_path in tree.getroot().findall("./datasources/datasource/connection/named-connections/named-connection/connection[@class='textscan']"):
            extract_name = extract_path.attrib['directory']#.split('/')[-1]
            extract_path.attrib['directory'] = f"{folder_path}/Data"

        try:
            with open (f"{folder_path}/{folder}.twb", "wb") as files :                                             
                tree.write(files)

            utilsc.convert_to_twbx(f"{folder_path}")
            print("twbx created successfully")

            # -------------------------------------------------- Getting Serial Numbers to upload to SF
            sn_df = uncovered_data_filtered[['Serial Number','Bk Sales Account Id Int','Branch Primary Name']]
            sn_df['USER_ID'] = user
            sn_df['SALES_LEVEL_1_NAME'] = theater
            sn_df['SALES_LEVEL_2_NAME'] = fields_df['Lvl2 (Region)'][idx]
            sn_df['COMPASS_REQ_ID'] = req_id
            sn_df['COMPLETION_QUARTER'] = f"FY{date_created.year + fy.get(qs.get(date_created.month))}Q{qs.get(date_created.month)}"
            sn_df['COMPLETION_DATE'] = date_created
            sn_df.rename(columns={'Serial Number':'SERIAL_NUMBER','Bk Sales Account Id Int':'CUSTOMER_ID','Branch Primary Name':'CUSTOMER_NAME'},inplace=True)
            sn_df["COMPLETION_DATE"] = sn_df["COMPLETION_DATE"].astype(str)
            sn_df['ACCOUNT_IDENTIFIER'] = id_type
            sn_df = sn_df[['USER_ID','SERIAL_NUMBER','CUSTOMER_ID','CUSTOMER_NAME','ACCOUNT_IDENTIFIER','SALES_LEVEL_1_NAME','SALES_LEVEL_2_NAME','COMPASS_REQ_ID','COMPLETION_QUARTER','COMPLETION_DATE']]
            sns_df = pd.concat([sns_df,sn_df])                
        except: print(bcolors.FAIL + "Error" + bcolors.ENDC) """

1/3: SFDCUS-112586 - twbx created successfully
2/3: MICROSOFTEMEAR-112590 - twbx created successfully
3/3: AvisBudgetGro-115463 - Error
Wall time: 673 ms


# Uploading URLs

In [6]:
folders_id = {
    "US COMMERCIAL": "fd7772ee-058d-463a-bd0a-4f65542801a4", # Americas
    "GLOBAL ENTERPRISE SEGMENT": "e57405f3-ca6c-4bc2-ad00-6e9e4eaddca3",
    "LATIN AMERICA": "11411241-08ec-488e-b5e5-b858051464a5",
    "CANADA": "2c7d703c-ccd7-4ebb-9a80-493efec42b37",
    "AMERICAS_SP": "001fdd45-0e78-4c94-8529-f541a58903b0",
    "US PS Market Segment": "bc2b9643-6dab-4659-967c-4c7faa1c9fbd",
    "ANZ AREA": "dc644422-3a51-4940-9e0c-5b9eeba4e938", # APJC
    "ASEAN_AREA": "486500cf-bab3-42f2-8cf6-912e0db03056",
    "GREATER_CHINA": "48eaf322-742a-4b72-a750-fe87a18a3e46",
    "INDIA_AREA": "3eda13f8-ce22-40fc-89e1-cca9f69318ee",
    "JAPAN__": "18c5b29f-d451-4546-b924-0f9964d98c95",
    "ROK_AREA": "0f98151c-0030-4f4b-90fc-a785b444cea1",
    "APJ_SP": "517cddf4-032f-4dc0-9569-23ce801ebd33",
    "EMEAR_GERMANY": "d11b1067-045d-42cb-a4c8-67805e4523a4", # EMEAR
    "EMEAR_SP": "35a5ac5e-1654-421e-bb75-d8c95a9408e3",
    "EMEAR-SOUTH": "7e91e72f-18bf-4b3b-8b72-7b964140743b",
    "EMEAR-NORTH": "34b13798-5c47-4bcf-b3d1-917d802d511f",
    "EMEAR-UKI": "268fdf46-d3ee-4623-ace9-97bb32ccc328",
    "EMEAR_MEA": "95dcde09-a6fc-4fb1-b7a8-69e1dc4775e8",
    "EMEAR-CENTRAL": "038970b3-8ea2-4a9a-86e8-6bd470350656"
}

fields_df2 = fields_df.copy()
fields_df2['folder_id'] = fields_df2['Lvl2 (Region)'].apply(lambda x : folders_id.get(x,''))
#fields_df2['folder_id'] = '2fffed65-9c48-4ac3-9c96-21b9c2b8ca8f'
fields_df2['project_name'] = folders_list
fields_df2['project_url'] = fields_df2['project_name'].apply(lambda x: utilsc.get_url(x))
fields_df2[['Customer Name','Lvl2 (Region)','folder_id','project_name','project_url']]

,Customer Name,Lvl2 (Region),folder_id,project_name,project_url
0,SFDC US,AMERICAS_SP,001fdd45-0e78-4c94-8529-f541a58903b0,CR_Americas_112586_SFDCUS_SAV ID_2022_3_10,https://cx-tableau-stage.cisco.com/#/site/Comp...
1,MICROSOFT EMEAR,AMERICAS_SP,001fdd45-0e78-4c94-8529-f541a58903b0,CR_Americas_112590_MICROSOFTEMEAR_SAV ID_2022_...,https://cx-tableau-stage.cisco.com/#/site/Comp...
2,Avis Budget Group,EMEAR-UKI,268fdf46-d3ee-4623-ace9-97bb32ccc328,CR_EMEAR_115463_AvisBudgetGro_SAV ID_2022_3_10,https://cx-tableau-stage.cisco.com/#/site/Comp...


In [7]:
%%time
# ---------------------------------------------------------

import os
import datetime

from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying
import time

current_path = os.getcwd().replace("\\","/")

#date = datetime.datetime.today()
#date = date.date()

month = datetime.datetime.today().strftime("%B")

config = {
    "tableau_server": {
        'server': 'https://cx-tableau-stage.cisco.com',
        'api_version': '3.13',
        'personal_access_token_name': personal_access_token_name,
        'personal_access_token_secret': personal_access_token_secret,
        'site_name': 'Compass',
        'site_url': 'Compass'
    }
}

conn = TableauServerConnection(config,env='tableau_server')

for idx in range(len(fields_df2)):
    conn.sign_in()
    project = fields_df2['project_name'][idx]
    
    if project != "N/A":
        name = fields_df2['Customer Name'][idx]

    # -------------------------------------------------------------------    

        try: 

            response = conn.publish_workbook(
                project_id=fields_df2['folder_id'][idx],
                workbook_file_path=f"{current_path}/CR/{month}/{date}/{project}.twbx",
                workbook_name="{}".format(project),
                #workbook_views_to_hide=['QA','Package Info'],
                #hide_view_flag=True
            )

            if (response.status_code != 201):    
                print(bcolors.FAIL + bcolors.BOLD + "Coverage Report has an error and has not been published"+bcolors.ENDC)
                raise AssertionError()

            else : print(f"{idx+1}/{len(fields_df2)}:The {name} The Coverage Report has been published")

        except Exception as e: 
            print(bcolors.FAIL + bcolors.BOLD + str(e)+bcolors.ENDC)
            pass

        time.sleep(3)
        conn.sign_out()


1/3:The SFDC US The Coverage Report has been published
2/3:The MICROSOFT EMEAR The Coverage Report has been published
3/3:The Avis Budget Group The Coverage Report has been published
Wall time: 26.5 s


In [8]:
for link in fields_df2['project_url']:
    if link != "N/A":
        webbrowser.open(link)          
    print(link)

https://cx-tableau-stage.cisco.com/#/site/Compass/views/CR_Americas_112586_SFDCUS_SAVID_2022_3_10/Overview?iframeSizedToWindow=true&%3Aembed=y&%3AshowAppBanner=false&%3Adisplay_count=no&%3AshowVizHome=no&%3Atabs=no&%3Aorigin=viz_share_link&%3Atoolbar=yes
https://cx-tableau-stage.cisco.com/#/site/Compass/views/CR_Americas_112590_MICROSOFTEMEAR_SAVID_2022_3_10/Overview?iframeSizedToWindow=true&%3Aembed=y&%3AshowAppBanner=false&%3Adisplay_count=no&%3AshowVizHome=no&%3Atabs=no&%3Aorigin=viz_share_link&%3Atoolbar=yes
https://cx-tableau-stage.cisco.com/#/site/Compass/views/CR_EMEAR_115463_AvisBudgetGro_SAVID_2022_3_10/Overview?iframeSizedToWindow=true&%3Aembed=y&%3AshowAppBanner=false&%3Adisplay_count=no&%3AshowVizHome=no&%3Atabs=no&%3Aorigin=viz_share_link&%3Atoolbar=yes


In [9]:
%%time
# Uploading Serial Numbers to Snowflake
try:
    utilsc.upload_data_to_sf(sns_df,user)
    print(f"{len(sns_df)} Serial Numbers uploaded")
    print('Uploaded time: ', datetime.datetime.now())
except: print(bcolors.FAIL + "Error" + bcolors.ENDC)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
2479 Serial Numbers uploaded
Uploaded time:  2022-03-10 14:33:37.911741
Wall time: 12.5 s
